<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/lopezbec/Traffic_Accident_Detection/blob/master/Yolov3_DeepSort/GPU/Implementation_Object_tracker.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

### Housekeeping Notes:

Before you  start working with this notebook, you need to first "mount" your Google Drive in the runtime's Virtual Machine. This will allow you to read and save files from your own Google Drive. 

To do this, you need to provide the path of your Google Drive for  the folder of this assignment. Once you provide this information, run this code cell below and follow the instructions. 


In [ ]:


import os
from google.colab import drive 

drive.mount('/content/gdrive',force_remount=True)

Working_Directory = 'Shared drives/GitHub FONDOCYT/Yolov3_DeepSort/GPU' #@param {type:"string"}
wd="/content/gdrive/"+Working_Directory
os.chdir(wd)


dirpath = os.getcwd()
print("current directory is : " + dirpath)





Mounted at /content/gdrive
current directory is : /content/gdrive/Shared drives/GitHub FONDOCYT/Yolov3_DeepSort/GPU


Before we begin, we need to install all the requirements

In [ ]:
#installing requirements
!pip install -r requirements-gpu.txt
from IPython.display import clear_output
#clear_output()
print("All Requirements were installed!")

Run this cell to download the weight files.

In [ ]:
#Downloading the weigts
!wget https://pjreddie.com/media/files/yolov3.weights -O weights/yolov3.weights

**Loading the weight files** 

In [ ]:
# yolov3
!python load_weights.py
#clear_output()
print("Weights were loaded!")

#**Runing the tracker**
You can run the tracker setting the video input and the output path for the resulting video.

Note: for this case the resulting video is allways going to be a .avi file.

Everytime the tracker is excecuted it is going to create/update a ".json" file containing:


*   Frame: the number of frame where the detection is made from.
*   Track_id: an identification given to every object detected in the video.
*   Class: the class of object detected.
*   Center: this is going to be a set of coordinates x and y pointing to the center of the object detected in every case.
*   Detection_box: this is a set of tow pairs of coordinates x and y, pointing to the starting and ending point for the rectangle drawned around every detection, this is going to be the next way (start_x, start_y,end_x,endy).


In [ ]:
# yolov3 on video
!python object_tracker.py --video ./data/video/test.mp4 --output ./data/video/results.avi
#clear_output()
print('''
        Video object tracking is done, go to /data/video
        to see the resulting video...
''')